### Imports and Convenience Functions

In [34]:
import warnings
import numpy as np
import phe as paillier
from sonar.contracts import ModelRepository,Model
from sonar.ipfs import IPFS
from syft.he.paillier.keys import KeyPair
from syft.nn.linear import LinearClassifier
from sklearn.datasets import load_diabetes

def get_balance(account):
    return repo.web3.fromWei(repo.web3.eth.getBalance(account),'ether')

warnings.filterwarnings('ignore')

## Setup Settings

In [35]:

# this is the addresas
my_wallet_address = '0xc6E1D4501D3f354E89F048F096db4424168682a9'

### Setting up the Experiment

In [2]:
# for the purpose of the simulation, we're going to split our dataset up amongst
# the relevant simulated users

diabetes = load_diabetes()
y = diabetes.target
X = diabetes.data

validation = (X[0:5],y[0:5])
anonymous_diabetes_users = (X[6:],y[6:])

# we're also going to initialize the model trainer smart contract, which in the
# real world would already be on the blockchain (managing other contracts) before
# the simulation begins

andreas_repo = '0xd60e1a150b59a89a8e6e6ff2c03ffb6cb4096205'
stefan_repo = '0xf0DE3D665D777d12609D0524c89C6456A6658E5E'



# ATTENTION: copy paste the correct address (NOT THE DEFAULT SEEN HERE) from truffle migrate output.
repo = ModelRepository(andreas_repo, account=my_wallet_address,ipfs=IPFS(host='127.0.0.1'), web3_host='localhost') # blockchain hosted model repository


Connected to OpenMined ModelRepository:0xd60e1a150b59a89a8e6e6ff2c03ffb6cb4096205


## Step 1: Cure Diabetes Inc Initializes a Model and Provides a Bounty

In [3]:
pubkey,prikey = KeyPair().generate(n_length=1024)
diabetes_classifier = LinearClassifier(desc="DiabetesClassifier",n_inputs=10,n_labels=1)

In [4]:
initial_error = diabetes_classifier.evaluate(validation[0],validation[1])

In [5]:
diabetes_classifier.encrypt(pubkey)

Linear Model (10,1): DiabetesClassifier

In [6]:
diabetes_model = Model(owner='0xc6E1D4501D3f354E89F048F096db4424168682a9',
                       syft_obj = diabetes_classifier,
                       bounty = 0.00000000001,
                       initial_error = initial_error,
                       target_error = 10000
                      )

In [7]:
model_id = repo.submit_model(diabetes_model)

In [14]:
len(repo)

11

## Step 2: An Anonymous Patient Downloads the Model and Improves It

In [15]:
model_id = 10
model = repo[model_id]

In [16]:
repo[model_id].submit_gradient(my_wallet_address,anonymous_diabetes_users[0][0],anonymous_diabetes_users[0][0])

## Step 3: Cure Diabetes Inc. Evaluates the Gradient 

In [17]:
repo[model_id]

Desc:DiabetesClassifier
Owner:0xc6e1d4501d3f354e89f048f096db4424168682a9
Bounty:9.999999E-12
Initial Error:21802517
Best Error:None
Target Error:10000
Model ID:10
Num Grads:0

In [18]:
old_balance = get_balance(my_wallet_address)
print(old_balance)

3.709448782750000015


In [21]:
grad = model[0]

In [22]:
grad.id

2

In [25]:
model = repo[model_id]

In [26]:
new_error = model.evaluate_gradient(my_wallet_address,repo[model_id][0],prikey,pubkey,validation[0],validation[1])

In [27]:
new_error

21802481

In [30]:
new_balance = get_balance(my_wallet_address)
incentive = new_balance - old_balance
print(incentive)

-0.001333360000000000


## Step 4: Rinse and Repeat

In [31]:
model

Desc:DiabetesClassifier
Owner:0xc6e1d4501d3f354e89f048f096db4424168682a9
Bounty:9.999999E-12
Initial Error:21802517
Best Error:None
Target Error:10000
Model ID:10
Num Grads:1

In [33]:
# for i,(addr, input, target) in enumerate(anonymous_diabetics):
#     try:
        
#         model = repo[model_id]
        
#         # patient is doing this
#         model.submit_gradient(addr,input,target)
        
#         # Cure Diabetes Inc does this
#         old_balance = get_balance(addr)
#         new_error = model.evaluate_gradient(cure_diabetes_inc,model[i+1],prikey,pubkey,validation[0],validation[1],alpha=2)
#         print("new error = "+str(new_error))
#         incentive = round(get_balance(addr) - old_balance,5)
#         print("incentive = "+str(incentive))
#     except:
#         "Connection Reset"